In [ ]:
"""
At the time of this notebook, we are lacking 
(1) OG script for the matching
(2) indication of program size limitations (how many students can take a given course before backup choice)
Goal: 
- Edit: <Explore the data and determine the demand of the various programs> Determine demands overall
<> Determine the number of programs that exceed a demand of assumed 30 max capacity per allocated classroom
- Are there any unfillable requests? Essentially, programs that are too highly demanded so that students 
    are put in a program outside of their preferences? 
"""

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [ ]:
# Initial choice is first school listed in "Career Fairs Planning.xlsx": Blackman MS/HS
# NOTE: some high schools are fed by two MS, like Riverdale 
# read in POS offerings 
POS_path = "../YouScienceData/Career Fairs Planning.xlsx"
POS_df = pd.read_excel(POS_path,"Blackman MS_Blackman HS")
POS_df.head()

In [ ]:
POS_df.POS[0:15]

In [ ]:
# reading in Blackman MS student roster with preferences
# NOTE: each student will have 3 entries ranked with their 1st-2nd-3rd choices
roster_path = "../YouScienceData/YouScience_cluster_advising_Blackman Middle School.csv"
roster_df = pd.read_csv(roster_path)
roster_df.head()

In [ ]:
len(roster_df)

In [ ]:
# Pruning to relevant features
pri_df = roster_df[roster_df.fit_rank == 1]
pri_df.aptitude_fit.value_counts().plot.bar()

In [ ]:
pri_df.interest_fit.value_counts().plot.bar()

In [ ]:
apt = set(pri_df.aptitude_fit)
print(len(apt), apt)
interest = set(pri_df.interest_fit)
print(len(interest), interest)

In [ ]:
for col in apt:
    pri_df.loc[pri_df.aptitude_fit == col, "apt_" + col] = 1
for col in interest:
    pri_df.loc[pri_df.interest_fit == col, "int_" + col] = 1

pri_df.replace(np.nan,0,inplace=True)


In [ ]:
pri_df.head()

In [ ]:
cols = list(pri_df.columns)
cooc_cols = []
for col in cols:
    if ("apt_" in col) or ("int_" in col):
        cooc_cols.append(col)

#cooc_cols 

In [ ]:
cooc_pre = pri_df[cooc_cols].astype(int)
coocc = cooc_pre.T.dot(cooc_pre)
np.fill_diagonal(coocc.values, 0)
coocc.head()

In [ ]:
apt_cols = []
int_cols = []
for col in cooc_cols:
    if "apt_" in col:
        apt_cols.append(col)
    else:
        int_cols.append(col)

In [ ]:
# fig, ax = plt.subplots(figsize=(18,7))
# sns.heatmap(coocc,annot=coocc.values, ax=ax)

In [ ]:
# need to determine how aptitude and interests align
demands = coocc[apt_cols]
demands.drop(apt_cols, axis = 0, inplace=True)
demands

In [ ]:
fig, ax = plt.subplots(figsize=(18,7))
sns.heatmap(demands, annot=demands.values, ax=ax)